In [0]:
%pip install file:/Workspace/Users/koen.koopen@datasciencelab.nl/marvelous-databricks-course-koenkoopen/dist/hotel_reservation-0.2.12-py3-none-any.whl
%restart_python

In [0]:
from pyspark.sql import SparkSession

from hotel_reservation.utils import configure_logging

logger = configure_logging("Hotel Reservations")

# Initialize Spark session
spark = SparkSession.builder.appName("HotelReservations").getOrCreate()

# Set the catalog and schema context
spark.sql("USE CATALOG koen_dev")
spark.sql("USE gold_hotel_reservations")

# Read data using SQL query
df = spark.sql("SELECT * FROM hotel_reservations")

# Show the first 10 rows
df.show()

In [0]:
from hotel_reservation.config import ProjectConfig
from hotel_reservation.preprocessing import DataProcessor

# Load configuration
config = ProjectConfig.from_yaml(config_path="../project_config.yml")

logger.info("Configuration loaded")

# COMMAND ----------
# Initialize DataProcessor
data_processor = DataProcessor(df, config, spark)

data_processor.preprocess()

In [0]:
df_train, df_test = data_processor.split_data()

data_processor.save_to_catalog(df_train, spark, table_name="train_dataset")
data_processor.save_to_catalog(df_test, spark, table_name="test_dataset")
# X_train = df_train.drop("booking_status")
# y_train = df_train.select("booking_status")

# X_test = df_test.drop("booking_status")
# y_test = df_test.select("booking_status")

# Enable change data feed (only once!)
logger.info("Enable change data feed")
data_processor.enable_change_data_feed()